In [1]:
import os
import time
import numpy as np
import soundfile as sf
import scipy
import csv
import scipy.misc
import imageio
import shutil
import sys
import datetime
import random
import collections
import pickle
import math

from random import randint

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
# set the memory usage
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_config))

from keras.layers import MaxPooling1D, Dropout, AveragePooling2D,LSTM,Reshape, Bidirectional, Add
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization,Conv1D
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import losses, optimizers
from keras.layers.pooling import GlobalMaxPooling1D

Using TensorFlow backend.


In [27]:
frames_per_sample = 200

def create_melspec_train(path):
    train_frames=[]
    train_labels=[]
    with open('event_list_devtrain_babycry.csv', 'rt') as train_file:
        reader = list(csv.reader(train_file, delimiter='\n'))
        
        for row,i in zip(reader,range(5000)):
            fields= str(row).split('\\t')
            
            temp_train_frames =[]
            temp_train_labels = []
                    
            if(len(fields)>1):
                if(fields[1] and fields[2]):
                    file_name = fields[0].replace("[\'", "").replace("\']", "")
                    melspec = imageio.imread(path + str(i) + '.png')
                    print(file_name)
                    event_start_index = int(np.round(float(fields[1])/30 * 1305))
                    event_finish_index = int(np.round(float(fields[2])/30 * 1305))
                    event_length = event_finish_index - event_start_index 
                    event_length = min(199,event_length)
                    event_finish_index = event_start_index + event_length
                    
                    print(event_start_index)
                    print(event_finish_index)
                    print(event_length)
                    #calculating sample start and finish indices, randomly
                    sample_start = randint(max(0,event_start_index - (200-event_length)),event_start_index)
                    sample_finish = min(1305,sample_start+frames_per_sample)
                    
                    print('before')
                    print(sample_start)
                    print(sample_finish)
                    sample_length = sample_finish - sample_start 
                    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
                        sample_start = sample_start - (frames_per_sample - sample_length)
                    
                    print('after')
                    print(sample_start)
                    print(sample_finish)
                    
                    temp1 = np.array(melspec)
                    temp_train_frames = temp1[:,sample_start:sample_finish]
                    
                    print(np.array(temp_train_frames).shape)
                    for ind in range(sample_start,event_start_index):
                        temp_train_labels.append(0)
                    for ind in range(event_start_index,event_finish_index):
                        temp_train_labels.append(1)
                    for ind in range(event_finish_index,sample_finish):
                        temp_train_labels.append(0)
                    
                    print('temp_train_labels.shape is: '+str(np.array(temp_train_labels).shape))   
                    temp2 = np.array(temp_train_frames)
                    temp2 = temp2.reshape(128,200)
                    train_frames.append(temp2)
                    
                    temp3 = np.array(temp_train_labels)
                    temp3 = temp3.reshape(200)
                    train_labels.append(temp3)
                    
                    print(np.array(train_frames).shape)
                    print(np.array(train_labels).shape)
            if(i%500==0):
                print('Reading files: '+str(i))
    
    train_file.close()
    train_data = {'train_frames':train_frames,
                 'train_labels':train_labels}
    return train_data

In [28]:
file_path = 'Datasets_legMel_44100_128/train_5000_babycry_Prob1_44100_128/'
d={}
d = create_melspec_train(file_path)

mixture_devtrain_babycry_000_440c5d9e8827bc5d9a9a687f2347fdbf.wav
263
389
126
before
226
426
after
226
426
(128, 200)
temp_train_labels.shape is: (200,)
(1, 128, 200)
(1, 200)
Reading files: 0
mixture_devtrain_babycry_001_87bed4da86b79d4512044cf1101c176a.wav
691
809
118
before
682
882
after
682
882
(128, 200)
temp_train_labels.shape is: (200,)
(2, 128, 200)
(2, 200)
mixture_devtrain_babycry_002_5a46c17510540c98060862d4404bb87a.wav
827
983
156
before
803
1003
after
803
1003
(128, 200)
temp_train_labels.shape is: (200,)
(3, 128, 200)
(3, 200)
mixture_devtrain_babycry_003_efcd68fc8f4b0d4d6c31a122105707b6.wav
289
338
49
before
223
423
after
223
423
(128, 200)
temp_train_labels.shape is: (200,)
(4, 128, 200)
(4, 200)
mixture_devtrain_babycry_004_297b08bc60329179f54da862acbdbe50.wav
1148
1286
138
before
1102
1302
after
1102
1302
(128, 200)
temp_train_labels.shape is: (200,)
(5, 128, 200)
(5, 200)
mixture_devtrain_babycry_005_f1de9141d7e8eb1f3910beb4e1502524.wav
450
507
57
before
376
576
afte

mixture_devtrain_babycry_054_c5f7ab24a27162d578d60e6b2e9abd49.wav
952
1091
139
before
942
1142
after
942
1142
(128, 200)
temp_train_labels.shape is: (200,)
(55, 128, 200)
(55, 200)
mixture_devtrain_babycry_055_0d86b2ec539c14456de53a477b3e77b6.wav
258
386
128
before
199
399
after
199
399
(128, 200)
temp_train_labels.shape is: (200,)
(56, 128, 200)
(56, 200)
mixture_devtrain_babycry_056_a8cff213f2a1899f5d2644f7656b052c.wav
227
288
61
before
224
424
after
224
424
(128, 200)
temp_train_labels.shape is: (200,)
(57, 128, 200)
(57, 200)
mixture_devtrain_babycry_057_57cde4f4a7b6964ba5ed39f5ecd7f22f.wav
150
349
199
before
150
350
after
150
350
(128, 200)
temp_train_labels.shape is: (200,)
(58, 128, 200)
(58, 200)
mixture_devtrain_babycry_058_61d36537ea01a1f6b605011364cc26a0.wav
827
983
156
before
812
1012
after
812
1012
(128, 200)
temp_train_labels.shape is: (200,)
(59, 128, 200)
(59, 200)
mixture_devtrain_babycry_059_804fc94437df5dd5781c360d27c10c4f.wav
1148
1286
138
before
1114
1305
after
110

before
981
1181
after
981
1181
(128, 200)
temp_train_labels.shape is: (200,)
(101, 128, 200)
(101, 200)
mixture_devtrain_babycry_101_9ff7f39f54bbf327d57b869717b47cee.wav
267
366
99
before
183
383
after
183
383
(128, 200)
temp_train_labels.shape is: (200,)
(102, 128, 200)
(102, 200)
mixture_devtrain_babycry_102_87226b79b2cfc5b15d1bc4a6e3a60f94.wav
970
1169
199
before
969
1169
after
969
1169
(128, 200)
temp_train_labels.shape is: (200,)
(103, 128, 200)
(103, 200)
mixture_devtrain_babycry_103_c2be31a37e5214a61792a351b1d281fb.wav
295
369
74
before
233
433
after
233
433
(128, 200)
temp_train_labels.shape is: (200,)
(104, 128, 200)
(104, 200)
mixture_devtrain_babycry_104_411ac5f6d2f63ee871bec077d8cc5a0f.wav
860
889
29
before
744
944
after
744
944
(128, 200)
temp_train_labels.shape is: (200,)
(105, 128, 200)
(105, 200)
mixture_devtrain_babycry_105_1f16296939a9858fcd0f0b2a36e96786.wav
258
386
128
before
206
406
after
206
406
(128, 200)
temp_train_labels.shape is: (200,)
(106, 128, 200)
(106, 2

mixture_devtrain_babycry_147_c0375f52810d96475579e661aa5ef7e2.wav
1012
1118
106
before
985
1185
after
985
1185
(128, 200)
temp_train_labels.shape is: (200,)
(148, 128, 200)
(148, 200)
mixture_devtrain_babycry_148_67edb302f1d00f1596f9a4e626e73727.wav
213
313
100
before
113
313
after
113
313
(128, 200)
temp_train_labels.shape is: (200,)
(149, 128, 200)
(149, 200)
mixture_devtrain_babycry_149_346dbe7d3e947621ec44aafed95b1958.wav
107
178
71
before
68
268
after
68
268
(128, 200)
temp_train_labels.shape is: (200,)
(150, 128, 200)
(150, 200)
mixture_devtrain_babycry_150_99826ef313cbec7882d4d6714ab6f63e.wav
315
396
81
before
290
490
after
290
490
(128, 200)
temp_train_labels.shape is: (200,)
(151, 128, 200)
(151, 200)
mixture_devtrain_babycry_151_bb56578c0cf7d1cfc868734ae6ad1124.wav
289
338
49
before
213
413
after
213
413
(128, 200)
temp_train_labels.shape is: (200,)
(152, 128, 200)
(152, 200)
mixture_devtrain_babycry_152_9638a34b2ae9fbd98afaf6003eaa6af4.wav
774
882
108
before
770
970
after
77

(230, 128, 200)
(230, 200)
mixture_devtrain_babycry_232_fd9995719cc47c7e052aac6ad318a204.wav
195
274
79
before
125
325
after
125
325
(128, 200)
temp_train_labels.shape is: (200,)
(231, 128, 200)
(231, 200)
mixture_devtrain_babycry_233_9e8854090f6138aa33c59aef8fbf04f0.wav
897
975
78
before
854
1054
after
854
1054
(128, 200)
temp_train_labels.shape is: (200,)
(232, 128, 200)
(232, 200)
mixture_devtrain_babycry_234_59d29f87877aa0fe65eec9203cecdf10.wav
546
687
141
before
541
741
after
541
741
(128, 200)
temp_train_labels.shape is: (200,)
(233, 128, 200)
(233, 200)
mixture_devtrain_babycry_235_a0e057bce3ba53306b2152aa695bd064.wav
1202
1284
82
before
1196
1305
after
1105
1305
(128, 200)
temp_train_labels.shape is: (200,)
(234, 128, 200)
(234, 200)
mixture_devtrain_babycry_236_c8fe0ec1c65549500fd5ca175f5c121e.wav
994
1046
52
before
977
1177
after
977
1177
(128, 200)
temp_train_labels.shape is: (200,)
(235, 128, 200)
(235, 200)
mixture_devtrain_babycry_237_afa0d2ed9d49c17039867d2a6b451601.wav


...

mixture_devtrain_babycry_4889_6d61da352ac7e17d75799f5761373497.wav
698
897
199
before
698
898
after
698
898
(128, 200)
temp_train_labels.shape is: (200,)
(4781, 128, 200)
(4781, 200)
mixture_devtrain_babycry_4890_ea0a881b31c22ff6abb0f6828138f872.wav
998
1169
171
before
998
1198
after
998
1198
(128, 200)
temp_train_labels.shape is: (200,)
(4782, 128, 200)
(4782, 200)
mixture_devtrain_babycry_4891_e4567801e7883dd476b9e344222d428b.wav
1
200
199
before
0
200
after
0
200
(128, 200)
temp_train_labels.shape is: (200,)
(4783, 128, 200)
(4783, 200)
mixture_devtrain_babycry_4892_9a3e5fa85347811c758760008471b6c7.wav
362
473
111
before
320
520
after
320
520
(128, 200)
temp_train_labels.shape is: (200,)
(4784, 128, 200)
(4784, 200)
mixture_devtrain_babycry_4893_3572ef9c3ef92a48da3b7c57e6b21129.wav
6
128
122
before
4
204
after
4
204
(128, 200)
temp_train_labels.shape is: (200,)
(4785, 128, 200)
(4785, 200)
mixture_devtrain_babycry_4894_ccf6e0d93b1ee5036393c8bfbfbf3100.wav
267
366
99
before
254
454
a

(4826, 200)
mixture_devtrain_babycry_4936_a9f1780d0e625129c89aa56d5e1e5bef.wav
691
809
118
before
673
873
after
673
873
(128, 200)
temp_train_labels.shape is: (200,)
(4827, 128, 200)
(4827, 200)
mixture_devtrain_babycry_4937_5e53bca4a7be44970d604eed40411346.wav
333
432
99
before
283
483
after
283
483
(128, 200)
temp_train_labels.shape is: (200,)
(4828, 128, 200)
(4828, 200)
mixture_devtrain_babycry_4938_71a87aa45cfdecc37ed4fa3dd505f86b.wav
934
1053
119
before
871
1071
after
871
1071
(128, 200)
temp_train_labels.shape is: (200,)
(4829, 128, 200)
(4829, 200)
mixture_devtrain_babycry_4939_63089cc2f629e11eb4dd817f9149c5ee.wav
267
366
99
before
196
396
after
196
396
(128, 200)
temp_train_labels.shape is: (200,)
(4830, 128, 200)
(4830, 200)
mixture_devtrain_babycry_4940_3ce2382141fe43a37ea378906a7d799d.wav
726
880
154
before
689
889
after
689
889
(128, 200)
temp_train_labels.shape is: (200,)
(4831, 128, 200)
(4831, 200)
mixture_devtrain_babycry_4941_c9bed16685002a84dc80faeecad752d6.wav
367
4

(4873, 128, 200)
(4873, 200)
mixture_devtrain_babycry_4984_821f889f8dff97dd7e12a5d19151123a.wav
594
688
94
before
511
711
after
511
711
(128, 200)
temp_train_labels.shape is: (200,)
(4874, 128, 200)
(4874, 200)
mixture_devtrain_babycry_4985_5d08bdc76382b639aa96ffc90e36b30d.wav
1061
1209
148
before
1013
1213
after
1013
1213
(128, 200)
temp_train_labels.shape is: (200,)
(4875, 128, 200)
(4875, 200)
mixture_devtrain_babycry_4986_e0cf84145f1c59556f2fe8cf6135aa75.wav
827
983
156
before
819
1019
after
819
1019
(128, 200)
temp_train_labels.shape is: (200,)
(4876, 128, 200)
(4876, 200)
mixture_devtrain_babycry_4987_620fe6872978b12b8891ac4611e980cb.wav
1090
1177
87
before
1067
1267
after
1067
1267
(128, 200)
temp_train_labels.shape is: (200,)
(4877, 128, 200)
(4877, 200)
mixture_devtrain_babycry_4988_05e8dc393476ed64256df09a661f63ef.wav
145
209
64
before
22
222
after
22
222
(128, 200)
temp_train_labels.shape is: (200,)
(4878, 128, 200)
(4878, 200)
mixture_devtrain_babycry_4989_573710aedb51049f2

(4918, 128, 200)
(4918, 200)
mixture_devtrain_babycry_5030_67a4b49f456daf9ed31f47d66c6e66cc.wav
362
473
111
before
354
554
after
354
554
(128, 200)
temp_train_labels.shape is: (200,)
(4919, 128, 200)
(4919, 200)
mixture_devtrain_babycry_5031_2e46968898c6daa6d758f9b87dec5bc5.wav
353
487
134
before
350
550
after
350
550
(128, 200)
temp_train_labels.shape is: (200,)
(4920, 128, 200)
(4920, 200)
mixture_devtrain_babycry_5032_666146804d97516c8eb8e5c5e3512b62.wav
677
788
111
before
614
814
after
614
814
(128, 200)
temp_train_labels.shape is: (200,)
(4921, 128, 200)
(4921, 200)
mixture_devtrain_babycry_5033_a1a214bc15a5c9d217e0129453982a0b.wav
73
272
199
before
72
272
after
72
272
(128, 200)
temp_train_labels.shape is: (200,)
(4922, 128, 200)
(4922, 200)
mixture_devtrain_babycry_5034_c79d1e1b7c3509d819dc0712d97cc078.wav
934
1053
119
before
909
1109
after
909
1109
(128, 200)
temp_train_labels.shape is: (200,)
(4923, 128, 200)
(4923, 200)
mixture_devtrain_babycry_5035_ea619d629da4a4ff1cd3915601

(4967, 128, 200)
(4967, 200)
mixture_devtrain_babycry_5080_b6ecb02cfdae63ca60d11cd0ba2bdde0.wav
28
75
47
before
25
225
after
25
225
(128, 200)
temp_train_labels.shape is: (200,)
(4968, 128, 200)
(4968, 200)
mixture_devtrain_babycry_5081_b28d57608185d29d5eaa78f34fb637e3.wav
227
288
61
before
125
325
after
125
325
(128, 200)
temp_train_labels.shape is: (200,)
(4969, 128, 200)
(4969, 200)
mixture_devtrain_babycry_5083_a5df825b837a07762f5d6d68558b21cb.wav
6
128
122
before
2
202
after
2
202
(128, 200)
temp_train_labels.shape is: (200,)
(4970, 128, 200)
(4970, 200)
mixture_devtrain_babycry_5084_0a9b25e24dbcbb5ed86038ea1443a32d.wav
246
391
145
before
194
394
after
194
394
(128, 200)
temp_train_labels.shape is: (200,)
(4971, 128, 200)
(4971, 200)
mixture_devtrain_babycry_5085_46cc330b0ef777c0194d328a463aa0a1.wav
457
505
48
before
420
620
after
420
620
(128, 200)
temp_train_labels.shape is: (200,)
(4972, 128, 200)
(4972, 200)
mixture_devtrain_babycry_5086_b553bc1f875e5ba8c32841f6b086c195.wav
37

In [29]:
np.array(d['train_frames']).shape

(5000, 128, 200)

In [30]:
train_frames = np.array(d['train_frames'])
train_labels = np.array(d['train_labels'])

In [31]:
f = open('Datasets_legMel_44100_128/train_5000_babycry_Prob1_44100_128_framelevel_calssifier.pkl','wb')
pickle.dump(d,f)
f.close()

In [32]:
frames_per_sample = 200

def create_melspec_val(path):
    val_frames=[]
    val_labels=[]
    with open('MyCode/data/TUT-rare-sound-events-2017-development/devtrain_3000_2/2d5cfbf13394fc98749052ea7d64893f/meta/event_list_devtrain_babycry.csv', 'rt') as val_file:
        reader = list(csv.reader(val_file, delimiter='\n'))
        
        for row,i in zip(reader,range(1000)):
            fields= str(row).split(',')
            
            temp_val_frames =[]
            temp_val_labels = []
                    
            if(len(fields)>1):
                if(fields[1] and fields[2]):
                    file_name = fields[0].replace("[\'", "").replace("\']", "")
                    melspec = imageio.imread(path + str(i) + '.png')
                    print(file_name)
                    event_start_index = int(np.round(float(fields[1])/30 * 1305))
                    event_finish_index = int(np.round(float(fields[2])/30 * 1305))
                    event_length = event_finish_index - event_start_index 
                    event_length = min(199,event_length)
                    event_finish_index = event_start_index + event_length
                    
                    print(event_start_index)
                    print(event_finish_index)
                    
                    #calculating sample start and finish indices, randomly
                    sample_start = randint(max(0,event_start_index - (200-event_length)),event_start_index)
                    sample_finish = min(1305,sample_start+frames_per_sample)
                    
                    print('before')
                    print(sample_start)
                    print(sample_finish)
                    sample_length = sample_finish - sample_start 
                    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
                        sample_start = sample_start - (frames_per_sample - sample_length)
                    
                    print('after')
                    print(sample_start)
                    print(sample_finish)
                    
                    temp1 = np.array(melspec)
                    temp_val_frames = temp1[:,sample_start:sample_finish]
                    
                    print(np.array(temp_val_frames).shape)
                    for ind in range(sample_start,event_start_index):
                        temp_val_labels.append(0)
                    for ind in range(event_start_index,event_finish_index):
                        temp_val_labels.append(1)
                    for ind in range(event_finish_index,sample_finish):
                        temp_val_labels.append(0)
                    
                    print('temp_val_labels.shape is: '+str(np.array(temp_val_labels).shape))   
                    temp2 = np.array(temp_val_frames)
                    temp2 = temp2.reshape(128,200)
                    val_frames.append(temp2)
                    
                    temp3 = np.array(temp_val_labels)
                    temp3 = temp3.reshape(200)
                    val_labels.append(temp3)
                    
                    print(np.array(val_frames).shape)
                    print(np.array(val_labels).shape)
            if(i%500==0):
                print('Reading files: '+str(i))
    
    val_file.close()
    val_data = {'val_frames':val_frames,
                 'val_labels':val_labels}
    return val_data

In [33]:
val_file_path = 'Datasets_legMel_44100_128/val_1000_babycry_44100_128/'
v={}
v = create_melspec_val(val_file_path)

Reading files: 0
mixture_devtrain_babycry_001_3ea98985c271accb95d0b6a24a185692.wav
677
752
before
565
765
after
565
765
(128, 200)
temp_val_labels.shape is: (200,)
(1, 128, 200)
(1, 200)
mixture_devtrain_babycry_004_d016e96174162784fa4be41a4271b86b.wav
210
342
before
149
349
after
149
349
(128, 200)
temp_val_labels.shape is: (200,)
(2, 128, 200)
(2, 200)
mixture_devtrain_babycry_007_8df55c0ce813e8d50f418861e0ffea61.wav
1001
1119
before
956
1156
after
956
1156
(128, 200)
temp_val_labels.shape is: (200,)
(3, 128, 200)
(3, 200)
mixture_devtrain_babycry_008_a9f9b2e10baa7ec338c3364ba448c29f.wav
252
319
before
136
336
after
136
336
(128, 200)
temp_val_labels.shape is: (200,)
(4, 128, 200)
(4, 200)
mixture_devtrain_babycry_009_ea93f27aabb9deacb9a40c363fe20e13.wav
1113
1251
before
1106
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(5, 128, 200)
(5, 200)
mixture_devtrain_babycry_010_0cb70934c7bbbb84bc3b8f431cf93cbd.wav
343
489
before
311
511
after
311
511
(128, 200)
temp_val_

mixture_devtrain_babycry_116_d982587d74dbfb79f42be3651762cfd8.wav
751
868
before
741
941
after
741
941
(128, 200)
temp_val_labels.shape is: (200,)
(57, 128, 200)
(57, 200)
mixture_devtrain_babycry_118_41386671f7559437c1c0e666fbeb2a31.wav
930
1052
before
854
1054
after
854
1054
(128, 200)
temp_val_labels.shape is: (200,)
(58, 128, 200)
(58, 200)
mixture_devtrain_babycry_120_6cd903ec61216a15b6abec6643417350.wav
1124
1188
before
999
1199
after
999
1199
(128, 200)
temp_val_labels.shape is: (200,)
(59, 128, 200)
(59, 200)
mixture_devtrain_babycry_121_c1dba933857f2928be74bb2acb4eaa1c.wav
19
108
before
4
204
after
4
204
(128, 200)
temp_val_labels.shape is: (200,)
(60, 128, 200)
(60, 200)
mixture_devtrain_babycry_123_2ad20135621d0d1d6c290824f73e6aec.wav
22
74
before
3
203
after
3
203
(128, 200)
temp_val_labels.shape is: (200,)
(61, 128, 200)
(61, 200)
mixture_devtrain_babycry_125_479ec0cd39d4a7a5fc973d5cf067eb76.wav
899
987
before
881
1081
after
881
1081
(128, 200)
temp_val_labels.shape is: (2

(104, 128, 200)
(104, 200)
mixture_devtrain_babycry_209_91c24002c93819a797894efb62387e20.wav
1037
1107
before
1021
1221
after
1021
1221
(128, 200)
temp_val_labels.shape is: (200,)
(105, 128, 200)
(105, 200)
mixture_devtrain_babycry_214_15f9dd47d53a13accf8a9566b21b76a6.wav
1121
1238
before
1071
1271
after
1071
1271
(128, 200)
temp_val_labels.shape is: (200,)
(106, 128, 200)
(106, 200)
mixture_devtrain_babycry_215_2f669df94681f2f71dde9ffd2b766b70.wav
479
605
before
473
673
after
473
673
(128, 200)
temp_val_labels.shape is: (200,)
(107, 128, 200)
(107, 200)
mixture_devtrain_babycry_217_cc51e7d0de4bb37f8ec1491aeb005710.wav
600
674
before
600
800
after
600
800
(128, 200)
temp_val_labels.shape is: (200,)
(108, 128, 200)
(108, 200)
mixture_devtrain_babycry_218_5d1413fc9c43b56a08f423884f067942.wav
22
74
before
10
210
after
10
210
(128, 200)
temp_val_labels.shape is: (200,)
(109, 128, 200)
(109, 200)
mixture_devtrain_babycry_219_cd7a601925a75c0d0b7b3a006ffc5226.wav
368
448
before
361
561
after


(151, 128, 200)
(151, 200)
mixture_devtrain_babycry_315_f943d08dcdaa728291638f70342a3b36.wav
555
623
before
510
710
after
510
710
(128, 200)
temp_val_labels.shape is: (200,)
(152, 128, 200)
(152, 200)
mixture_devtrain_babycry_316_bafa5826bdd4e57440f5868602672109.wav
790
928
before
749
949
after
749
949
(128, 200)
temp_val_labels.shape is: (200,)
(153, 128, 200)
(153, 200)
mixture_devtrain_babycry_317_288786fa4e7157a44eae8c9b92e7c2f1.wav
1037
1107
before
948
1148
after
948
1148
(128, 200)
temp_val_labels.shape is: (200,)
(154, 128, 200)
(154, 200)
mixture_devtrain_babycry_323_99e4742580c58fd1dc7021ee4e0d2013.wav
74
193
before
0
200
after
0
200
(128, 200)
temp_val_labels.shape is: (200,)
(155, 128, 200)
(155, 200)
mixture_devtrain_babycry_324_0bcaff18450e654a846ad4eccf019f2e.wav
343
489
before
312
512
after
312
512
(128, 200)
temp_val_labels.shape is: (200,)
(156, 128, 200)
(156, 200)
mixture_devtrain_babycry_325_cc9aaa9a37b81fe8eae373ca0cd6fc27.wav
285
332
before
231
431
after
231
431
(

(235, 128, 200)
(235, 200)
mixture_devtrain_babycry_481_fc4e62a221a1818785c99a1ae211e609.wav
930
1052
before
891
1091
after
891
1091
(128, 200)
temp_val_labels.shape is: (200,)
(236, 128, 200)
(236, 200)
mixture_devtrain_babycry_483_7bbba808f0f6a9a62e6af82baa24ede0.wav
887
960
before
802
1002
after
802
1002
(128, 200)
temp_val_labels.shape is: (200,)
(237, 128, 200)
(237, 200)
mixture_devtrain_babycry_484_c5fec72ea6ebf1c6d11e323be436f5e9.wav
1066
1171
before
1012
1212
after
1012
1212
(128, 200)
temp_val_labels.shape is: (200,)
(238, 128, 200)
(238, 200)
mixture_devtrain_babycry_485_5dbee2c4da5d68f3967411ebaf70e8f6.wav
13
77
before
12
212
after
12
212
(128, 200)
temp_val_labels.shape is: (200,)
(239, 128, 200)
(239, 200)
mixture_devtrain_babycry_487_c55db4fb6fded34b7880e63ab0751c53.wav
1124
1188
before
1110
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(240, 128, 200)
(240, 200)
mixture_devtrain_babycry_488_da1475b7707cb7583cc0faa89e70192c.wav
498
567
before
380
580
a

mixture_devtrain_babycry_622_afba0d0d0e2481c5fe2971704f76d6f7.wav
1135
1246
before
1113
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(308, 128, 200)
(308, 200)
mixture_devtrain_babycry_624_87953a4e0677527c90205d4d0ffabbb6.wav
677
752
before
656
856
after
656
856
(128, 200)
temp_val_labels.shape is: (200,)
(309, 128, 200)
(309, 200)
mixture_devtrain_babycry_627_25a1b2e276df19c660914cc37882f6d4.wav
930
1052
before
908
1108
after
908
1108
(128, 200)
temp_val_labels.shape is: (200,)
(310, 128, 200)
(310, 200)
mixture_devtrain_babycry_629_22fa3dee61b9a2f4a903a8845c0314e4.wav
275
364
before
220
420
after
220
420
(128, 200)
temp_val_labels.shape is: (200,)
(311, 128, 200)
(311, 200)
mixture_devtrain_babycry_630_5d0884fe94e72e8857890c01764b20cd.wav
855
1009
before
816
1016
after
816
1016
(128, 200)
temp_val_labels.shape is: (200,)
(312, 128, 200)
(312, 200)
mixture_devtrain_babycry_631_d5b7b0296e92ca8aa973a606a4947bdc.wav
555
623
before
539
739
after
539
739
(128, 200)
temp

144
344
after
144
344
(128, 200)
temp_val_labels.shape is: (200,)
(373, 128, 200)
(373, 200)
mixture_devtrain_babycry_762_a9ef415f3f9400ce4510f6745419a36e.wav
730
841
before
653
853
after
653
853
(128, 200)
temp_val_labels.shape is: (200,)
(374, 128, 200)
(374, 200)
mixture_devtrain_babycry_765_ae7dfac3c3281d949711b5f5de53580f.wav
135
311
before
129
329
after
129
329
(128, 200)
temp_val_labels.shape is: (200,)
(375, 128, 200)
(375, 200)
mixture_devtrain_babycry_766_3f650dd867d4c148c2c35949515a3ca6.wav
55
147
before
48
248
after
48
248
(128, 200)
temp_val_labels.shape is: (200,)
(376, 128, 200)
(376, 200)
mixture_devtrain_babycry_767_5c148fccd1f510c40618845e17af3ce5.wav
605
675
before
508
708
after
508
708
(128, 200)
temp_val_labels.shape is: (200,)
(377, 128, 200)
(377, 200)
mixture_devtrain_babycry_768_a0ebbedb0a841821a12e9978b671f377.wav
1019
1218
before
1018
1218
after
1018
1218
(128, 200)
temp_val_labels.shape is: (200,)
(378, 128, 200)
(378, 200)
mixture_devtrain_babycry_769_80b7d

(434, 200)
mixture_devtrain_babycry_885_c0682734e85cbdd70297bb69fae3c869.wav
1121
1238
before
1064
1264
after
1064
1264
(128, 200)
temp_val_labels.shape is: (200,)
(435, 128, 200)
(435, 200)
mixture_devtrain_babycry_889_32e6e6955b33f1f27be941400a53e694.wav
453
610
before
426
626
after
426
626
(128, 200)
temp_val_labels.shape is: (200,)
(436, 128, 200)
(436, 200)
mixture_devtrain_babycry_890_3d6ab85579e349520f6a9eeb097a2926.wav
855
1009
before
811
1011
after
811
1011
(128, 200)
temp_val_labels.shape is: (200,)
(437, 128, 200)
(437, 200)
mixture_devtrain_babycry_891_8a9f04b1c8be5a0b9c1e9cc0c655873d.wav
225
320
before
130
330
after
130
330
(128, 200)
temp_val_labels.shape is: (200,)
(438, 128, 200)
(438, 200)
mixture_devtrain_babycry_893_e9141cf94430547691cde664efc77036.wav
368
506
before
351
551
after
351
551
(128, 200)
temp_val_labels.shape is: (200,)
(439, 128, 200)
(439, 200)
mixture_devtrain_babycry_894_bd14ca58e8a03815316b71b59aaa70ba.wav
1113
1251
before
1104
1304
after
1104
1304
(

mixture_devtrain_babycry_1006_88cd7548d26a93ad94ffaf2751d86557.wav
465
664
before
464
664
after
464
664
(128, 200)
temp_val_labels.shape is: (200,)
(489, 128, 200)
(489, 200)
mixture_devtrain_babycry_1007_6539be06178ec2255e57c5803e09de7c.wav
498
567
before
445
645
after
445
645
(128, 200)
temp_val_labels.shape is: (200,)
(490, 128, 200)
(490, 200)
mixture_devtrain_babycry_1008_cf4e68b364888c83ddad8bfdd39fa1ce.wav
672
812
before
640
840
after
640
840
(128, 200)
temp_val_labels.shape is: (200,)
(491, 128, 200)
(491, 200)
mixture_devtrain_babycry_1012_3998d4d4a5badabe9cee70c139ae2e33.wav
245
338
before
149
349
after
149
349
(128, 200)
temp_val_labels.shape is: (200,)
(492, 128, 200)
(492, 200)
mixture_devtrain_babycry_1016_9c5a52bf2de34fff217487924ec683af.wav
677
752
before
619
819
after
619
819
(128, 200)
temp_val_labels.shape is: (200,)
(493, 128, 200)
(493, 200)
mixture_devtrain_babycry_1018_6ab561cce57f966d138b6704d7f8b9b1.wav
605
675
before
587
787
after
587
787
(128, 200)
temp_val_l

In [34]:
val_frames = np.array(v['val_frames'])
val_labels = np.array(v['val_labels'])

In [35]:
f = open('Datasets_legMel_44100_128/val_1000_babycry_44100_128_framelevel_calssifier.pkl','wb')
pickle.dump(v,f)
f.close()

In [10]:
def get_model():
    
    
    inp = Input(shape=(128,200,1))
    x = Conv2D(16, (5,5), activation='relu', padding="valid")(inp)
    x = MaxPooling2D((5,5))(x)
    x = Dropout(rate=0.3)(x)
    
    
    
    x = Conv2D(64, (3,3), activation='relu', padding="valid")(x)
    x = MaxPooling2D((3,3))(x)
    x = Dropout(rate=0.3)(x)
    
    x = Conv2D(128, (3,3), activation='relu', padding="valid")(x)
    x = MaxPooling2D((3,3))(x)
    x = Dropout(rate=0.3)(x)
    
    x = Flatten()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(200, activation='sigmoid')(x)
    
    
    
    model = Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(lr=0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model


In [11]:
my_model = get_model()

In [12]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 200, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 196, 16)      416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 39, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 39, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 37, 64)        9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 12, 64)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 12, 64)         0         
__________

In [13]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=20, mode='auto')

In [ ]:
train_frames = np.array(train_frames).reshape(5000,128,200,1)
val_frames = np.array(val_frames).reshape(500,128,200,1)
train_labels = np.array(train_labels).reshape(5000,200)
val_labels = np.array(val_labels).reshape(500,200)

In [ ]:
my_model.fit(x=train_frames,y=train_labels,batch_size=200,epochs=150, 
             validation_data=(val_frames, val_labels), callbacks=[earlystop])